# Regrid ERA5 winds to JRA55 winds

In [ ]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe

from dask.distributed import Client

In [ ]:
client = Client(n_workers=7)
client

In [ ]:
ds_era5 = xr.open_dataset('/g/data/rt52/era5/single-levels/reanalysis/10u/1958/10u_era5_oper_sfc_19580101-19580131.nc')
ds_jra55 = xr.open_dataset('/g/data/qv56/replicas/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-4-0/atmos/3hrPt/uas/gr/v20190429/uas_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-4-0_gr_195801010000-195812312100.nc')

In [ ]:
bilin = xe.Regridder(ds_era5,ds_jra55, method="bilinear", periodic=True)
bilin.to_netcdf('/g/data/e14/rmh561/era5_to_jra55_regridder.nc')

In [ ]:
bilin = xe.Regridder(ds_era5,ds_jra55, method="bilinear", periodic=True, weights='/g/data/e14/rmh561/era5_to_jra55_regridder.nc')

In [ ]:
ds_era5 = ds_era5.sel(time=ds_jra55.time.sel(time=slice('1958-01-01','1958-01-31'))).load()

In [ ]:
%%time
ds_era5_jra55grid = bilin(ds_era5)

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(12,8))

ds_era5.u10.isel(time=0).plot(ax=axes[0][0],vmin=-20.,vmax=20.,cmap='RdBu_r')
axes[0][0].set_title('ERA5')
ds_jra55.uas.isel(time=0).plot(ax=axes[1][0],vmin=-20.,vmax=20.,cmap='RdBu_r')
axes[1][0].set_title('JRA55')
ds_era5_jra55grid.u10.isel(time=0).plot(ax=axes[0][1],vmin=-20.,vmax=20.,cmap='RdBu_r')
axes[0][1].set_title('ERA5 on JRA55 grid')
(ds_era5_jra55grid.u10 - ds_jra55.uas).isel(time=0).plot(ax=axes[1][1],vmin=-20.,vmax=20.,cmap='RdBu_r')
axes[1][1].set_title('ERA5 on JRA55 grid - JRA55')